In [2]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector, pos:Integer) => prediction(pos))

getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))


UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))

In [3]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.0


In [7]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model-rf2"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model-rf2")

val predictionsDF = sameModel.transform(inputDF)


inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model-rf2
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]
sameModel = pipeline_b1b68980d298
predictionsDF = [tweet: string, words: array<string> ... 5 more fields]


lastException: Throwable = null


[tweet: string, words: array<string> ... 5 more fields]

In [9]:
//Streaming

predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) => 
    batchDF
    .withColumn("negative_probability",getProbability($"probability",lit(0)))
    .withColumn("positive_probability",getProbability($"probability",lit(1)))
    .withColumn("positive", when($"predictedLabel"===0,1))
    .withColumn("negative", when($"predictedLabel"===1,1))
    .withColumn("timestamp", expr("current_timestamp"))
    .withWatermark("timestamp", "10 seconds")
    .groupBy(
      window($"timestamp", "10 seconds"))
    .agg(count($"*").as("tweets")
       ,sum("positive").as("positive")
       ,sum("negative").as("negative")
        )
    
   .selectExpr( "current_timestamp() as load_dttm"
              , "tweets"
              , "cast(positive/tweets*100.0 as int) as `positive,%`"
              , "cast(negative/tweets*100.0 as int) `negative,%`")
    .show(false)
    
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@36327560

+-----------------------+------+----------+----------+
|load_dttm              |tweets|positive,%|negative,%|
+-----------------------+------+----------+----------+
|2019-08-25 12:31:21.742|10    |20        |80        |
+-----------------------+------+----------+----------+

+-----------------------+------+----------+----------+
|load_dttm              |tweets|positive,%|negative,%|
+-----------------------+------+----------+----------+
|2019-08-25 12:31:31.146|18    |27        |72        |
+-----------------------+------+----------+----------+

